In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from openai import OpenAI
import gradio as gr

In [5]:
load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")

if openai_key and google_key:
    print("we good to go🤙")
else:
    print("check your api key")

we good to go🤙


In [10]:
openai = OpenAI()
googleai = OpenAI(api_key=google_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [11]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No Title Found!!"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [12]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [24]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
def system_prompt(tone):
    prompt_tone = f"You are an assistant with a {tone} tone that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Using your tone, Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."
    return prompt_tone

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
def stream_gpt(tone, company_name, url):
    messages = [
        {"role": "system", "content": system_prompt(tone)},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [19]:
def stream_gemini(tone, company_name, url):
    messages = [
        {"role": "system", "content": system_prompt(tone)},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ]
    stream = googleai.chat.completions.create(
        model="gemini-2.5-flash-preview-04-17",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [20]:
def stream_model(model, tone, company_name, url):
    if model=="GPT":
        result = stream_gpt(tone, company_name, url)
    elif model=="Gemini":
        result = stream_gemini(tone, company_name, url)
    else:
        raise ValueError("Unknown model")
    yield from result

In [29]:
brochure = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT"),
        gr.Dropdown(["Formal", "Sarcastic", "Snarky", "Snippy"], label="Select Tone", value="Formal"),
        gr.Textbox(label="Company Name"),
        gr.Textbox(label="Company Url")
    ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
brochure.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
